<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/PonchonSavarit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar
from scipy.optimize import root
from scipy.special import expit, logit
from plotly.subplots import make_subplots
from scipy.interpolate import Akima1DInterpolator

In [3]:
R=8.314
p=Props(['Benzene', 'Toluene'])

In [4]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [5]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [6]:
def bubbleP_NRTL(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT_NRTL(x, P):

    def f(T):
        return bubbleP_NRTL(x,T)[0]-P

    #mole-fraction weighted boiling points of each component at P
    #boiling points determined via Clausius Clapeyron, using the Hvap at the normal bp
    #for each component.  p.Hvap returns the heat of vaporization of all components for each
    #temperature if an array of temperatures is given.
    Tguess=np.dot(x,1/(1/p.Tbn-np.log(P/101325)*R/np.diagonal(p.Hvap(p.Tbn))))

    res=root_scalar(f, x0=Tguess, method='secant')
    if not(res.converged):
        return "FAIL", res
    T=root_scalar(f, x0=Tguess, method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [7]:
P=101325
x1s=np.linspace(0,1,101)
Ts=[]
y1s=[]
for x1 in x1s:
    T, (y1,y2) = bubbleT_NRTL(np.array([x1, 1-x1]), P)
    Ts.append(T)
    y1s.append(y1)

In [8]:
y1interp = Akima1DInterpolator(x1s, y1s)
x1interp = Akima1DInterpolator(y1s, x1s)

In [9]:
fig=make_subplots(rows=1,cols=1)
fig.add_scatter(x=x1s, y=y1s, mode='lines', line_color='blue', row=1,col=1)
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', line_color='green', row=1,col=1)
fig.update_layout(width=500, height=500, showlegend=False)

In [10]:
Hv=[]
Hl=[]
for T, x1, y1 in zip(Ts, x1s, y1s):
    Hv.append(p.Hv([y1,1-y1], T))
    Hl.append(p.Hl([x1,1-x1], T))

Hv = Akima1DInterpolator(y1s, Hv)
Hl = Akima1DInterpolator(x1s, Hl)

In [11]:
F = 1.
zF = 0.45
q=1.
R=3.
rD = 0.98  #fraction recovery of more volatile component in the distillate
xD = 0.99
D=rD*F*zF/xD
B = F - D
rB = 1-rD
xB=rB*F*zF/B

In [12]:
Qc = (R+1)*D*(Hv(xD)-Hl(xD))
deltaR = Qc/D + Hl(xD)

In [13]:
Qb  = Qc + B*Hl(xB) + D*Hl(xD)- F*Hl(zF)
deltaS = Hl(xB) - Qb/B

In [21]:
fig2=make_subplots(rows=1,cols=2)
z=np.linspace(0,1,51)
fig2.add_scatter(x=z, y=Hl(z), row=1,col=1)
fig2.add_scatter(x=z, y=Hv(z), row=1, col=1)
fig2.add_scatter(x=x1s, y=y1s, mode='lines', line_color='blue', row=1,col=2)
fig2.add_scatter(x=[0,1],y=[0,1], mode='lines', line_color='green', row=1,col=2)

y = xD

def rec_eq(y2,x):
    return (deltaR - Hv(y2))/(Hv(y2)- Hl(x)) - (xD - y2)/(y2 - x)

def strip_eq(y2,x):
    return (Hl(x)-deltaS)/(Hv(y2)-Hl(x)) - (x-xB)/(y2 - x)

xs=[xD]
ys=[xD]

for i in range(100):
    x = x1interp(y)
    xs.append(x)
    ys.append(y)

    fig2.add_scatter(x=[y, x], y=[Hv(y), Hl(x)], row=1, col=1, line_color='rgb(0,100,100)', mode='lines',
                line_dash='dot')
    if x < xB:
        break

    if x>zF:
        fig2.add_scatter(x=[xD, x], y=[deltaR, Hl(x)], row=1, col=1, line_color='rgb(100,100,100)', mode='lines')
        y=root_scalar(rec_eq, bracket=(x+1e-14,xD-1e-14), args=(x,)).root
    else:
        y=root_scalar(strip_eq, bracket=(x+1e-14,xD-1e-14), args=(x,)).root
        fig2.add_scatter(x=[xB, y], y=[deltaS, Hv(y)], row=1, col=1, line_color='rgb(100,100,100)', mode='lines')

    xs.append(x)
    ys.append(y)


fig2.add_scatter(x=xs, y=ys, mode='lines', line_color='orange', row=1,col=2)
fig2.add_scatter(x=xs[::2], y=ys[::2], mode='lines', line_color='pink', row=1,col=2)
fig2.update_layout(width=800,height=500,template='plotly_dark', showlegend=False)

In [15]:
p.HfIG

array([82880., 50170.])